In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/src')

import numpy as np
from deformation import *
from scipy.optimize import minimize, fmin_l_bfgs_b, Bounds

torchdeviceId = torch.device('cuda:0') 
torchdtype = torch.float64
torch.cuda.empty_cache()

### Read and prepare data

In [2]:
data = torch.load('../Data/amygdala.pt')
amyg_sour = data['Source']
amyg_tar = data['Target']

VS = torch.from_numpy(amyg_sour['x']).clone().to(dtype=torchdtype, device=torchdeviceId)
FS = torch.from_numpy(amyg_sour['G']).clone().to(dtype=torch.long, device=torchdeviceId)
VT = torch.from_numpy(amyg_tar['x']).clone().to(dtype=torchdtype, device=torchdeviceId)
FT = torch.from_numpy(amyg_tar['G']).clone().to(dtype=torch.long, device=torchdeviceId)

Source = mesh2var(VS,FS)
Target = mesh2var(VT,FT)

results = {'amyg_sour':amyg_sour, 'amyg_tar':amyg_tar}

### Set parameters

In [3]:
'''model parameters'''
options = {}
options['defo_kernel_size'] = 4.75
options['odemethod'] = 'rk4'
options['nb_euler_steps'] = 15
options['kernel_geom'] = 'gaussian'
options['kernel_grass'] = 'binet'
options['sig_geom'] = 3
options['sig_grass'] = 1.1
options['weight_varifold_dist'] = 10
#options['weight_meta'] = .00001

'''optimization parameters'''
opt_par = {'gtol': 1e-6, 'disp': True, 'step_size':1e-4, 
           'maxiter': 200, 'maxls': 50}

### Standard LDDMM varifold matching approach

In [4]:
'''model parameters'''
options = {}
options['defo_kernel_size'] = 4.75
options['odemethod'] = 'rk4'
options['nb_euler_steps'] = 15
options['kernel_geom'] = 'gaussian'
options['kernel_grass'] = 'binet'
options['sig_geom'] = 3
options['sig_grass'] = 1.1
options['weight_varifold_dist'] = 10
#options['weight_meta'] = .00001
options['model'] = 'pure_varifold'

'''optimization parameters'''
opt_par = {'gtol': 1e-6, 'disp': True, 'step_size':1e-4, 
           'maxiter': 200, 'maxls': 50}


energy_scipy = enr_scipy(Source,Target,**options) #Energy function
x0 = np.zeros(list2vec(Source).shape[0])

start = time.time()

res = GD_adapted(energy_scipy,x0,**opt_par)
reg_time = round(time.time() - start, 2)
_,_, Enr_dic = energy_scipy(res.x)


results['pure_varifold'] = {'res':res,'Enr_dic':Enr_dic,'options':options,'time':reg_time}
torch.save(results,'amygdala_var_result.pt')

iter:1 | fun:2.09005e+05 | grad norm:4.69232e+05 | step size:1.71875e-06 | # step size search 6
iter:2 | fun:1.97384e+05 | grad norm:5.35141e+05 | step size:1.89063e-06 | # step size search 0
iter:3 | fun:9.41953e+04 | grad norm:1.98275e+05 | step size:1.03984e-06 | # step size search 1
iter:4 | fun:8.50720e+04 | grad norm:1.37009e+05 | step size:1.14383e-06 | # step size search 0
iter:5 | fun:8.26407e+04 | grad norm:1.25816e+05 | step size:1.25821e-06 | # step size search 0
iter:6 | fun:7.77026e+04 | grad norm:3.28005e+04 | step size:6.92016e-07 | # step size search 1
iter:7 | fun:7.70731e+04 | grad norm:2.76153e+04 | step size:7.61218e-07 | # step size search 0
iter:8 | fun:7.65280e+04 | grad norm:2.50974e+04 | step size:8.37339e-07 | # step size search 0
iter:9 | fun:7.60255e+04 | grad norm:2.30231e+04 | step size:9.21073e-07 | # step size search 0
iter:10 | fun:7.55586e+04 | grad norm:2.11769e+04 | step size:1.01318e-06 | # step size search 0
iter:11 | fun:7.51237e+04 | grad norm:1

iter:86 | fun:6.86265e+04 | grad norm:6.17739e+03 | step size:1.38430e-06 | # step size search 0
iter:87 | fun:6.85945e+04 | grad norm:7.67700e+03 | step size:1.52273e-06 | # step size search 0
iter:88 | fun:6.85781e+04 | grad norm:1.23428e+04 | step size:1.67501e-06 | # step size search 0
iter:89 | fun:6.85370e+04 | grad norm:8.78903e+03 | step size:9.21253e-07 | # step size search 1
iter:90 | fun:6.85085e+04 | grad norm:7.66164e+03 | step size:1.01338e-06 | # step size search 0
iter:91 | fun:6.84822e+04 | grad norm:7.56628e+03 | step size:1.11472e-06 | # step size search 0
iter:92 | fun:6.84572e+04 | grad norm:8.33372e+03 | step size:1.22619e-06 | # step size search 0
iter:93 | fun:6.84374e+04 | grad norm:1.05623e+04 | step size:1.34881e-06 | # step size search 0
iter:94 | fun:6.84008e+04 | grad norm:5.78434e+03 | step size:7.41844e-07 | # step size search 1
iter:95 | fun:6.83808e+04 | grad norm:5.11771e+03 | step size:8.16028e-07 | # step size search 0
iter:96 | fun:6.83607e+04 | gr

iter:170 | fun:6.70005e+04 | grad norm:3.88810e+03 | step size:1.01358e-06 | # step size search 0
iter:171 | fun:6.69886e+04 | grad norm:3.85778e+03 | step size:1.11493e-06 | # step size search 0
iter:172 | fun:6.69760e+04 | grad norm:4.00371e+03 | step size:1.22643e-06 | # step size search 0
iter:173 | fun:6.69629e+04 | grad norm:4.49076e+03 | step size:1.34907e-06 | # step size search 0
iter:174 | fun:6.69513e+04 | grad norm:5.88477e+03 | step size:1.48398e-06 | # step size search 0
iter:175 | fun:6.69501e+04 | grad norm:9.75707e+03 | step size:1.63238e-06 | # step size search 0
iter:176 | fun:6.69270e+04 | grad norm:6.38155e+03 | step size:8.97806e-07 | # step size search 1
iter:177 | fun:6.69138e+04 | grad norm:5.27198e+03 | step size:9.87587e-07 | # step size search 0
iter:178 | fun:6.69023e+04 | grad norm:5.03766e+03 | step size:1.08635e-06 | # step size search 0
iter:179 | fun:6.68914e+04 | grad norm:5.38320e+03 | step size:1.19498e-06 | # step size search 0
iter:180 | fun:6.688

In [2]:
#results = torch.load('amygdala_result.pt')

In [3]:
#results.keys()

dict_keys(['amyg_sour', 'amyg_tar', 'pure_varifold', 'metapwm', 'metapf'])